In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/IIP1/Assignments/Second/ModelFull/'Final Model and Code HERE'

/content/drive/MyDrive/IIP1/Assignments/Second/ModelFull/Final Model and Code HERE


In [ ]:
# Installing the "Simple Transformer" library
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.2 

In [ ]:
# Importing the necessary libraries
import logging
import sklearn
import os
import requests
import json

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
# Getting training and testing datasets

os.mkdir('squad')
url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'

for file in ['train-v2.0.json', 'dev-v2.0.json']:
  res = requests.get(f'{url}{file}') #make request
  with open(f'squad/{file}', 'wb') as f:
    for chunk in res.iter_content(chunk_size=4):
      f.write(chunk)

In [ ]:
# Preparing a testing dataset (6425 samples)
with open("squad/dev-v2.0.json", "rb") as f:
  squad_dict = json.load(f)

test = []
for i in range(17,35):
  test = test + squad_dict['data'][i]['paragraphs']

In [ ]:
print(test[0])

{'qas': [{'question': 'What is it called when people in society rebel against laws they think are unfair?', 'id': '57271c235951b619008f860b', 'answers': [{'text': 'Civil disobedience', 'answer_start': 130}, {'text': 'Civil disobedience', 'answer_start': 130}, {'text': 'Civil disobedience', 'answer_start': 130}, {'text': 'Civil disobedience', 'answer_start': 130}, {'text': 'Civil disobedience', 'answer_start': 130}], 'is_impossible': False}, {'question': 'What is an example of major civil disobedience in South Africa?', 'id': '57271c235951b619008f860c', 'answers': [{'text': 'apartheid', 'answer_start': 497}, {'text': 'fight against apartheid', 'answer_start': 483}, {'text': 'the fight against apartheid', 'answer_start': 479}, {'text': 'the fight against apartheid', 'answer_start': 479}, {'text': 'Singing Revolution to bring independence to the Baltic countries', 'answer_start': 554}], 'is_impossible': False}, {'question': 'What was the the movement called that brought Baltic countries i

In [ ]:
# https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

model_args = {
    #general arguments
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    # Maximum sequence length the model will support.
    "max_seq_length": 128,
    # The number of predictions given per question.
    "n_best_size":3,
    "train_batch_size": 128,
    "num_train_epochs": 50,
    "evaluate_during_training": True,
    "use_cached_eval_features": True,
    "eval_batch_size": 64,
    # Evaluate the model for each 800 optimization steps
    "evaluate_during_training_steps": 800,
}

In [ ]:
# Use the pretrained model form Hugging Face
model = QuestionAnsweringModel(
    "bert","bert-base-cased", args = model_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

---

---
3. **Test the model.**

In [ ]:
# Test the model
result, texts = model.eval_model(test, test_em_score=sklearn.metrics.accuracy_score)

Running Evaluation:   0%|          | 0/102 [00:00<?, ?it/s]

In [ ]:
print(result)
# correct-> number of correct predictions
# similar-> number of similar predictions
# incorrect -> number of incorrect predictions
# test_em_score -> correct/(correct + similar + incorrect)

{'correct': 0, 'similar': 3957, 'incorrect': 2468, 'test_em_score': 0.0, 'eval_loss': -0.09788866604075712}


In [ ]:
print(texts)
# detailed results

{'correct_text': {}, 'similar_text': {'57271c235951b619008f860b': {'truth': 'Civil disobedience', 'predicted': 'in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem', 'question': 'What is it called when people in society rebel against laws they think are unfair?'}, '5a66413dc2b11c001a425ec7': {'truth': '', 'predicted': 'in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem', 'question': 'What is the definition of Germany adopting a Communist government?'}, '5a66413dc2b11c001a425ec8': {'truth': '', 'predicted': 'in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem', 'question': 'What did the Velvet revolution hope to accomplish in India?'}, '5a66413dc2b11c001a425ec9': {'truth': '', 'predicted': 'in the 1919 Revolution. Civil disobedience is one of the many ways people have rebelled against what they deem', 'question'

---

---
4. **Answer questions with the model.**

In [ ]:
# The list of questions to be answered

questions = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ]
    },
    {
        "context": "Bidirectional Encoder Representations from Transformers (BERT) is a family of language models introduced in 2018 by researchers at Google.",
        "qas": [
            {
                "question": "When was BERT introduced?",
                "id": "1",
            }
        ]
    },
    {
        "context": "Bidirectional Encoder Representations from Transformers (BERT) is a family of language models introduced in 2018 by researchers at Google.",
        "qas": [
            {
                "question": "What color is sky?",
                "id": "2",
            }
        ]
    },
    {
        "context": "The 2022 FIFA World Cup was a professional association football tournament, the world championship for national football teams organized by FIFA. The 22nd edition of the FIFA World Cup, it took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan.[A] This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot climate,[B] the event was held during November and December.[C] It was held over a reduced time frame of 29 days with 64 matches played in eight venues across five cities. Argentine captain Lionel Messi was voted the tournament's best player, winning the Golden Ball.",
        "qas": [
            {
                "question": "Who was voted the tournament's best player?",
                "id": "3",
            }
        ]
    },
    {
        "context": "The 2022 FIFA World Cup was a professional association football tournament, the world championship for national football teams organized by FIFA. The 22nd edition of the FIFA World Cup, it took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World Cup to be held in the Arab world and Muslim world, and the second held entirely in Asia after the 2002 tournament in South Korea and Japan. Argentine captain Lionel Messi was voted the tournament's best player, winning the Golden Ball. [A] This tournament was the last with 32 participating teams, with the number of teams being increased to 48 for the 2026 edition. To avoid the extremes of Qatar's hot climate,[B] the event was held during November and December.[C] It was held over a reduced time frame of 29 days with 64 matches played in eight venues across five cities.",
        "qas": [
            {
                "question": "Who was voted the tournament's best player?",
                "id": "4",
            }
        ]
    },
    {
        "context": "Earth is the third planet from the Sun and the only place known in the universe where life has originated and found habitability.",
        "qas": [
            {
                "question": "What is Earth?",
                "id": "5",
            }
        ]
    },
    {
        "context": "Earth is the third planet from the Sun and the only place known in the universe where life has originated and found habitability.",
        "qas": [
            {
                "question": "Where has life originated?",
                "id": "6",
            }
        ]
    }
]


In [ ]:
# Answer questions with the model.

answers, probabilities = model.predict(questions)

print(answers)
print(probabilities)

add example index and unique id: 100%|██████████| 7/7 [00:00<00:00, 76858.97it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['empty']}, {'id': '1', 'answer': ['empty']}, {'id': '2', 'answer': ['empty']}, {'id': '3', 'answer': ['. The 22nd edition of the FIFA World Cup, it took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World', '. It was the first World', '. The 22nd edition']}, {'id': '4', 'answer': ['. The 22nd edition of the FIFA World Cup, it took place in Qatar from 20 November to 18 December 2022, after the country was awarded the hosting rights in 2010. It was the first World', ". Argentine captain Lionel Messi was voted the tournament's best player, winning", '. The 22nd edition']}, {'id': '5', 'answer': ['empty']}, {'id': '6', 'answer': ['empty']}]
[{'id': '0', 'probability': [0.5997488946822385]}, {'id': '1', 'probability': [0.6483550374749418]}, {'id': '2', 'probability': [0.7036512946448463]}, {'id': '3', 'probability': [0.3148584437431872, 0.31439756296131355, 0.31088648339246056]}, {'id':